In [ ]:
%matplotlib notebook

**Problem 1a**

In [ ]:
import tifffile as tiff
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML


# Load the TIFF stack
video_path = '/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small-motion.tif'
frames = tiff.imread(video_path)
num_frames = frames.shape[0]

print(f"Loaded {num_frames} frames, each of shape {frames.shape[1:]}")

# Part A: Play the video using matplotlib animation
fig, ax = plt.subplots()
im = ax.imshow(frames[0], cmap='gray', animated=True)

def update(frame_idx):
    im.set_array(frames[frame_idx])
    ax.set_title(f"Frame {frame_idx}")
    return [im]

ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=100, blit=True)

# Display the animation in notebook
mpl.rcParams['animation.embed_limit'] = 100  # allow bigger animations
HTML(ani.to_jshtml())
#plt.show()

**Problem 1b**

In [ ]:
import scipy.ndimage
import matplotlib.pyplot as plt

frame1 = frames[10]
frame2 = frames[260]  # First drastic change observed

# Normalize frames for correlation
f1 = (frame1 - np.mean(frame1)) / np.std(frame1)
f2 = (frame2 - np.mean(frame2)) / np.std(frame2)

# Define max shift range
max_shift = 10
correlations = np.zeros((2 * max_shift + 1, 2 * max_shift + 1))

# Compute correlation for each spatial shift
for dx in range(-max_shift, max_shift + 1):
    for dy in range(-max_shift, max_shift + 1):
        shifted = scipy.ndimage.shift(f2, shift=(dy, dx), mode='nearest')
        correlation = np.sum(f1 * shifted) / f1.size
        correlations[dy + max_shift, dx + max_shift] = correlation

# Find peak correlation shift
peak = np.unravel_index(np.argmax(correlations), correlations.shape)
peak_dy = peak[0] - max_shift
peak_dx = peak[1] - max_shift
print(f"Maximum correlation at shift: dx={peak_dx}, dy={peak_dy}")

# Plot correlation heatmap
plt.imshow(correlations, extent=[-max_shift, max_shift, -max_shift, max_shift], cmap='hot')
plt.colorbar(label="Correlation")
plt.title("Correlation vs Spatial Shift")
plt.xlabel("X Shift")
plt.ylabel("Y Shift")
plt.annotate(f'Peak ({peak_dx},{peak_dy})', xy=(peak_dx, peak_dy), xytext=(peak_dx+1, peak_dy+1),
             arrowprops=dict(arrowstyle='->'), color='white', fontsize=10)
plt.scatter(peak_dx, peak_dy, color='blue', label="Peak")
plt.legend()
plt.show()

In [ ]:
print("Max correlation value:", np.max(correlations))
print("Correlation at peak (-2,2):", correlations[peak[0], peak[1]])

In [ ]:
import scipy.ndimage
import matplotlib.pyplot as plt

frame1 = frames[10]
frame2 = frames[330]  # First drastic change observed

# Normalize frames for correlation
f1 = (frame1 - np.mean(frame1)) / np.std(frame1)
f2 = (frame2 - np.mean(frame2)) / np.std(frame2)

# Define max shift range
max_shift = 10
correlations = np.zeros((2 * max_shift + 1, 2 * max_shift + 1))

# Compute correlation for each spatial shift
for dx in range(-max_shift, max_shift + 1):
    for dy in range(-max_shift, max_shift + 1):
        shifted = scipy.ndimage.shift(f2, shift=(dy, dx), mode='nearest')
        correlation = np.sum(f1 * shifted) / f1.size
        correlations[dy + max_shift, dx + max_shift] = correlation

# Find peak correlation shift
peak = np.unravel_index(np.argmax(correlations), correlations.shape)
peak_dy = peak[0] - max_shift
peak_dx = peak[1] - max_shift
print(f"Maximum correlation at shift: dx={peak_dx}, dy={peak_dy}")

# Plot correlation heatmap
plt.imshow(correlations, extent=[-max_shift, max_shift, -max_shift, max_shift], cmap='hot')
plt.colorbar(label="Correlation")
plt.title("Correlation vs Spatial Shift")
plt.xlabel("X Shift")
plt.ylabel("Y Shift")
plt.annotate(f'Peak ({peak_dx},{peak_dy})', xy=(peak_dx, peak_dy), xytext=(peak_dx+1, peak_dy+1),
             arrowprops=dict(arrowstyle='->'), color='white', fontsize=10)
plt.scatter(peak_dx, peak_dy, color='blue', label="Peak")
plt.legend()
plt.show()

print("Max correlation value:", np.max(correlations))
print("Correlation at peak (0,1):", correlations[peak[0], peak[1]])

**Problem 2a**

In [ ]:
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Load the motion-corrected calcium image video
file_path = '/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small.tif'
video = tiff.imread(file_path)

print(f"Video shape: {video.shape}  [frames, height, width]")

# Compute summary images
mean_img = np.mean(video, axis=0)
median_img = np.median(video, axis=0)
var_img = np.var(video, axis=0)

# Plot the images
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

axs[0].imshow(mean_img, cmap='plasma')
axs[0].set_title('Mean Image')
axs[0].axis('off')

axs[1].imshow(median_img, cmap='plasma')
axs[1].set_title('Median Image')
axs[1].axis('off')

axs[2].imshow(var_img, cmap='hot')
axs[2].set_title('Variance Image')
axs[2].axis('off')

plt.tight_layout()
plt.show()

**Answer**: 
**We computed summary images using the mean, median, and variance of pixel intensities across time. The mean image provides a general overview of fluorescence brightness but tends to blur out dynamic activity. The median image is ideally more robust to transient spikes and gives a cleaner view of consistently active regions. The variance image, in contrast, highlights pixels with high temporal fluctuation, making it effective at identifying neurons that exhibit strong activity over time.**

In [ ]:
diff_img = mean_img - median_img  
plt.imshow(diff_img, cmap='seismic')
plt.title("Mean - Median")
plt.colorbar()
plt.show()

**Answer**: 
**Since there wasn't a high visual difference between mean and median from the summary images. I experimented by plotting the change between the two sets. In such neurons, rare high-amplitude events increase the mean intensity while the median remains low. This contrast is visible in the form of bright white or red regions in the image. In contrast, neurons with consistent activity show little to no difference between their mean and median values and appear dark in this visualization.**

**Problem 2b**

In [ ]:
range_img = np.max(video, axis=0) - np.min(video, axis=0)
std_img = np.std(video, axis=0)

# Plot additional stats
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].imshow(range_img, cmap='inferno')
axs[0].set_title('Max - Min (Range)')
axs[0].axis('off')

axs[1].imshow(std_img, cmap='plasma')
axs[1].set_title('Standard Deviation')
axs[1].axis('off')

plt.tight_layout()
plt.show()

**Answer: A good summary statistic should capture both spatial localization and temporal activity. I also experimented with the range (max - min) and standard deviation. The range image revealed neurons with bursty or sporadic activity, while standard deviation provided a clearer representation of general fluctuation. These results confirm that temporal variability-based statistics are better for identifying functionally relevant regions.**

**Problem 3a**

In [ ]:
import cv2
import numpy as np
import tifffile as tiff

# Load and normalize summary image
summary_img = np.var(tiff.imread('/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small.tif'), axis=0)
summary_img_norm = (summary_img - np.min(summary_img)) / (np.max(summary_img) - np.min(summary_img))
summary_img_8bit = (summary_img_norm * 255).astype(np.uint8)

# Resize for display
display_img = cv2.resize(summary_img_8bit, (512, 512))
clicked_coords = []

# Callback
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(clicked_coords) < 5:
        clicked_coords.append((x, y))
        print(f"Clicked: {len(clicked_coords)} -> x={x}, y={y}")
        cv2.circle(display_img, (x, y), 3, (255, 0, 0), -1)
        cv2.imshow("Click 5 seed points", display_img)

cv2.namedWindow("Click 5 seed points")
cv2.setMouseCallback("Click 5 seed points", click_event)

# Manual polling loop — auto exit after 5 clicks
while True:
    cv2.imshow("Click 5 seed points", display_img)
    if len(clicked_coords) >= 5:
        break
    key = cv2.waitKey(10)
    if key == 27:  # ESC to exit early
        print("Manual exit")
        break

cv2.destroyAllWindows()

# Scale back to original coordinates
orig_h, orig_w = summary_img.shape
scale_x = orig_w / 512
scale_y = orig_h / 512
scaled_coords = [(int(x * scale_x), int(y * scale_y)) for (x, y) in clicked_coords]

print("Final scaled seed coordinates (x, y):", scaled_coords)

In [ ]:
from skimage.draw import disk
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Load and normalize summary image
summary_img = np.var(tiff.imread('/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small.tif'), axis=0)
summary_img_norm = (summary_img - np.min(summary_img)) / (np.max(summary_img) - np.min(summary_img))

# Seeds: scaled_coords from your OpenCV selection
# e.g., scaled_coords = [(356, 56), (344, 269), (416, 399), (173, 484), (117, 146)]
roi_masks = []
radius = 11  # radius of circular ROI in pixels

for i, (x, y) in enumerate(scaled_coords):
    mask = np.zeros_like(summary_img_norm, dtype=bool)
    rr, cc = disk((y, x), radius, shape=summary_img.shape)
    mask[rr, cc] = True
    roi_masks.append(mask)

    # Visualize the mask
    masked_img = np.zeros_like(summary_img_norm)
    masked_img[mask] = summary_img_norm[mask]

    print(f"ROI {i+1}: seed=({x},{y}), mask pixels={np.sum(mask)}")

    plt.figure(figsize=(5, 5))
    plt.imshow(masked_img, cmap='plasma')
    plt.title(f'Circular ROI {i + 1}')
    plt.axis('off')
    plt.show()

**Problem 3b**

In [ ]:
import matplotlib.pyplot as plt
from skimage.measure import find_contours

# Use the same summary image used for ROI creation
summary_img = np.var(tiff.imread('/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small.tif'), axis=0)
summary_img_norm = (summary_img - np.min(summary_img)) / (np.max(summary_img) - np.min(summary_img))

# Overlay each ROI mask on the summary image
for idx, mask in enumerate(roi_masks):
    plt.figure(figsize=(5, 5))
    plt.imshow(summary_img_norm, cmap='gray')
    contours = find_contours(mask, level=0.5)

    for contour in contours:
        plt.plot(contour[:, 1], contour[:, 0], color='red', linewidth=1.5)

    plt.title(f'ROI {idx+1} - Mask Overlay')
    plt.axis('off')
    plt.show()

**Problem 4a**

In [ ]:
# Load the original motion-corrected video
video = tiff.imread('/Users/mananbhatt/Downloads/TEST_MOVIE_00001-small.tif')  # shape: [T, H, W]

# Extract time-traces
traces = []

for idx, mask in enumerate(roi_masks):
    trace = [np.mean(frame[mask]) for frame in video]
    traces.append(trace)

    # Plot time-trace
    plt.figure(figsize=(6, 2))
    plt.plot(trace, label=f'ROI {idx+1}')
    plt.title(f'Time-Trace for ROI {idx+1}')
    plt.xlabel('Frame')
    plt.ylabel('Fluorescence')
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.animation as animation
import matplotlib as mpl
from IPython.display import HTML
import matplotlib.pyplot as plt

roi_index = 2
trace = traces[roi_index]
mask = roi_masks[roi_index]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))
im = ax1.imshow(video[0], cmap='gray')
contour_lines = []

# Draw ROI contours on the first frame (initially)
contours = find_contours(mask, level=0.5)
for contour in contours:
    line, = ax1.plot(contour[:, 1], contour[:, 0], color='red', linewidth=1.2)
    contour_lines.append(line)

# Plot trace
line_trace, = ax2.plot([], [], lw=2)
ax2.set_xlim(0, len(trace))
ax2.set_ylim(np.min(trace), np.max(trace))
ax2.set_title(f"Time-trace for ROI {roi_index+1}")
ax2.set_xlabel("Frame")
ax2.set_ylabel("Fluorescence")

def update(frame_num):
    im.set_array(video[frame_num])
    line_trace.set_data(np.arange(frame_num + 1), trace[:frame_num + 1])
    return [im, line_trace] + contour_lines

ani = animation.FuncAnimation(fig, update, frames=len(video), interval=10, blit=True)
mpl.rcParams['animation.embed_limit'] = 100  # allow bigger animations
HTML(ani.to_jshtml())

**Answer: Yes, the extracted time-traces can reflect the actual neural activity captured in the calcium imaging video — provided that the ROIs used are accurate and well-localized. These traces are intended to represent the temporal dynamics of fluorescence changes within each ROI, which in turn correlate with neuronal calcium activity.**

**Problem 5**

In [ ]:
# video shape: [T, H, W] → reshape to [pixels, time]
T, H, W = video.shape
X = video.reshape(T, H * W).T  # shape: [pixels, time]
print(f"Data matrix shape: {X.shape}")  # (pixels, time)
print(f"Video shape: {video.shape}  [frames, height, width]")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Set number of PCA components
n_components = 20

# Apply PCA
pca = PCA(n_components=n_components)
pca_components = pca.fit_transform(X)  
pca_maps = pca_components.T.reshape(n_components, H, W)

In [ ]:
# Plot 10 PCA spatial components 
fig, axs = plt.subplots(1, 10, figsize=(15, 3))
for i in range(10):
    axs[i].imshow(pca_maps[i], cmap='gray')
    axs[i].set_title(f'PC {i+1}')
    axs[i].axis('off')
plt.suptitle('PCA Spatial Maps')
plt.tight_layout()
plt.show()

In [ ]:
# Plot 20 PCA spatial components 
fig, axs = plt.subplots(2, 10, figsize=(20, 5))
axs = axs.ravel()

for i in range(n_components):
    axs[i].imshow(pca_maps[i], cmap='gray')
    axs[i].set_title(f'PC {i+1}')
    axs[i].axis('off')

plt.suptitle('PCA Spatial Maps (20 Components)')
plt.tight_layout()
plt.show()

**Answer:**

1. PCA Spatial maps often represent mixtures of multiple neurons or global patterns.
2. Time-traces that capture directions of high variance.


Upon changing the components: 
1. Fewer components capture global fluctuations.
2. Increasing components introduces more detail but can include noise or redundant features.


In [ ]:
from sklearn.decomposition import NMF

n_components = 20
nmf = NMF(n_components=n_components, init='random', random_state=0, max_iter=500)
W = nmf.fit_transform(X)           
H_matrix = nmf.components_         
nmf_maps = W.T.reshape(n_components, video.shape[1], video.shape[2])  

In [ ]:
# Plot NMF spatial maps for 10 components
fig, axs = plt.subplots(1, 10, figsize=(20, 3))
for i in range(10):
    axs[i].imshow(nmf_maps[i], cmap='hot')
    axs[i].set_title(f'NMF {i+1}')
    axs[i].axis('off')
plt.suptitle('NMF Spatial Maps')
plt.tight_layout()
plt.show()

In [ ]:
# Plot NMF spatial maps for 20 components
fig, axs = plt.subplots(2, 10, figsize=(15, 3))
axs = axs.ravel()
for i in range(20):
    axs[i].imshow(nmf_maps[i], cmap='hot')
    axs[i].set_title(f'NMF {i+1}')
    axs[i].axis('off')
plt.suptitle('NMF Spatial Maps')
plt.tight_layout()
plt.show()

**Answer:**

Yes, NMF (Non-negative Matrix Factorization) is highly dependent on the rank of decomposition


1. Sparse spatial maps.
2. Time-traces that are strictly non-negative.

Upon changing the number of components: 

1. Too few components may combine multiple neurons.
2. Too many components may result in overfitting or component splitting.


In [ ]:
from sklearn.decomposition import FastICA

n_components = 20
video_height, video_width = video.shape[1], video.shape[2]

ica = FastICA(n_components=n_components, random_state=0)
ica_components = ica.fit_transform(X)  # shape: (pixels, n_components)
ica_maps = ica_components.T.reshape(n_components, video_height, video_width)


In [ ]:

# Plot 10 ICA spatial maps
fig, axs = plt.subplots(1, 10, figsize=(15, 3))
for i in range(10):
    axs[i].imshow(ica_maps[i], cmap='grey')
    axs[i].set_title(f'ICA {i+1}')
    axs[i].axis('off')
plt.suptitle('ICA Spatial Maps')
plt.tight_layout()
plt.show()

In [ ]:
# Plot 20 ICA spatial maps
fig, axs = plt.subplots(2, 10, figsize=(20, 3))
axs = axs.ravel()
for i in range(n_components):
    axs[i].imshow(ica_maps[i], cmap='grey')
    axs[i].set_title(f'ICA {i+1}')
    axs[i].axis('off')
plt.suptitle('ICA Spatial Maps')
plt.tight_layout()
plt.show()

**Answer:**

1. Focused spatial maps that can align with individual neurons.
2. Time-traces that may resemble spike trains.

Effect of changing n_components:
1. Too few components may miss relevant neurons.
2. Too many components may extract noise or break apart true signals.

**Some trial efforts to further analyse what individual components of PCA, NMF, ICA are doing and representing.**

In [ ]:
print(np.max(pca_maps[0]))
print((pca_maps.shape))

In [ ]:
threshold = 5000  # tweak this as needed
binary_roi = pca_maps[0] > threshold

plt.imshow(summary_img_norm, cmap='gray')
plt.contour(binary_roi, colors='red')
plt.title("Extracted ROI from PCA Component")
plt.axis('off')
plt.show()

In [ ]:
threshold = 0.2  # tweak this as needed
binary_roi = nmf_maps[0] > threshold

plt.imshow(summary_img_norm, cmap='gray')
plt.contour(binary_roi, colors='red')
plt.title("Extracted ROI from NMF Component")
plt.axis('off')
plt.show()

In [ ]:
# X.T is shape [time, pixels]
ica = FastICA(n_components=n_components, random_state=0)
S = ica.fit_transform(X.T)  # shape: [time, components]
A = ica.mixing_             # shape: [pixels, components]

ica_spatial_maps = A.T.reshape(n_components, video_height, video_width)
ica_time_traces = S.T  # shape: [components, time]

# Time-trace plot
plt.figure(figsize=(10, 4))
for i in range(n_components):
    plt.plot(ica_time_traces[i], label=f'ICA {i+1}')
plt.xlabel("Frame")
plt.ylabel("Activity")
plt.title("ICA Component Time-Traces")
plt.legend()
plt.tight_layout()
plt.show()

# Example spatial map
plt.figure()
plt.imshow(ica_spatial_maps[0], cmap='gray')
plt.title("ICA Spatial Map 1")
plt.axis('off')
plt.show()

In [ ]:
from skimage.morphology import remove_small_objects
from skimage.measure import label

component_idx = 2  # Choose the one that looks neuron-like
spatial_map = ica_spatial_maps[component_idx]

# Normalize the map (optional but recommended)
spatial_map_norm = (spatial_map - np.min(spatial_map)) / (np.max(spatial_map) - np.min(spatial_map))

# Threshold
binary_mask = spatial_map_norm > 0.3  # adjust threshold between 0.2 – 0.5 based on image
binary_mask = remove_small_objects(binary_mask, min_size=30)

# Label regions (optional if you want the largest blob only)
labeled = label(binary_mask)


plt.imshow(spatial_map_norm, cmap='gray')
plt.contour(binary_mask, colors='red')
plt.title(f"ICA ROI {component_idx+1}")
plt.axis('off')
plt.show()

roi_trace = [np.mean(frame[binary_mask]) for frame in video]

plt.figure(figsize=(6, 2))
plt.plot(roi_trace)
plt.title(f"Fluorescence Trace of ICA ROI {component_idx+1}")
plt.xlabel("Frame")
plt.ylabel("Mean Intensity")
plt.tight_layout()
plt.show()